In [2]:
# !pip install --upgrade pip
# !pip install -q unsloth peft trl wandb evaluate jsonlines human-eval
# !pip install -q datasets transformers accelAerate einops tiktoken tqdm
# !pip uninstall xformers -y
# !pip install xformers
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import json
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments, EarlyStoppingCallback, AutoTokenizer
import wandb
from pathlib import Path
import re
import torch
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
import os
from tqdm import tqdm

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# Step 1: Load model and tokenizer with Unsloth (4-bit quantization)
model_id = "unsloth/Qwen2.5-Coder-3B-Instruct"
max_seq_length = 4096  # Adjust based on your needs (Qwen supports up to 32k, but for efficiency)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length=max_seq_length,
    dtype=None,  # Auto-detect (bfloat16 if supported)
    load_in_4bit=True,  # Enable 4-bit quantization
)
# Load tokenizer đúng của phiên bản INSTRUCT
tokenizer = AutoTokenizer.from_pretrained(
    "unsloth/Qwen2.5-Coder-3B-Instruct",
    trust_remote_code=True
)

==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

In [5]:
# 1️⃣ KHẮC PHỤC LỖI PAD TOKEN BUG
# -----------------------------------------------------
# Qwen Instruct đã có pad token tích hợp sẵn → dùng đúng pad token:
# - Không được sử dụng pad_token = eos_token
# -----------------------------------------------------
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token   # Trường hợp hiếm cho các model cũ
print("Pad token:", tokenizer.pad_token)

# -----------------------------------------------------

# Load dataset JSONL
dataset = Dataset.from_json("/content/drive/MyDrive/QwenNoQuen/training_cleaned.jsonl")


# 2️⃣ KHẮC PHỤC LỖI “UNTRAINED TOKENS”
# -----------------------------------------------------
# ❗ Với bản INSTRUCT → được phép dùng chat template
# ❗ KHÔNG được dùng chat template khi model là *base*
# -----------------------------------------------------
def format_chat(example):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful coding assistant. "
                "Complete the function based on the problem description and starter code."
            )
        },
        {
            "role": "user",
            "content": (
                f"Problem Description:\n{example['problem_description']}\n\n"
                f"Starter Code:\n{example['starter_code']}\n\n"
                f"Please complete the function."
            )
        },
        {
            "role": "assistant",
            "content": example["completion"]
        }
    ]

    # apply_chat_template: CHỈ DÙNG CHO BẢN INSTRUCT
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False  # Không thêm thừa <|im_start|>assistant
    )
    return {"text": formatted_text}


# Format toàn dataset
formatted_dataset = dataset.map(format_chat, num_proc=4)

# Kiểm tra 1 sample
print(formatted_dataset[0]["text"])

Pad token: <|PAD_TOKEN|>


Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=4):   0%|          | 0/2819 [00:00<?, ? examples/s]

<|im_start|>system
You are a helpful coding assistant. Complete the function based on the problem description and starter code.<|im_end|>
<|im_start|>user
Problem Description:
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
You may assume that each input would have exactly one solution, and you may not use the same element twice.
You can return the answer in any order.
 
Example 1:

Input: nums = [2,7,11,15], target = 9
Output: [0,1]
Explanation: Because nums[0] + nums[1] == 9, we return [0, 1].

Example 2:

Input: nums = [3,2,4], target = 6
Output: [1,2]

Example 3:

Input: nums = [3,3], target = 6
Output: [0,1]

 
Constraints:

2 <= nums.length <= 104
-109 <= nums[i] <= 109
-109 <= target <= 109
Only one valid answer exists.

 
Follow-up: Can you come up with an algorithm that is less than O(n2) time complexity?

Starter Code:
def twoSum(self, nums: List[int], target: int) -> List[int]:
    

Please complete th

In [6]:
# Step 3: Apply LoRA adapters via Unsloth
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank of LoRA adapters
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",  # Efficient checkpointing
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.3 patched 36 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [7]:
# Step 5: Set up training arguments
training_args = TrainingArguments(
    output_dir = "./output",
    num_train_epochs=1.3,  # Few epochs to avoid overfitting on 2800 samples
    per_device_train_batch_size=2,  # Adjust based on GPU memory
    gradient_accumulation_steps=8,  # Effective batch size 16
    learning_rate=2e-5,
    warmup_ratio=0.05,   # ~5% của total steps
    lr_scheduler_type="cosine",  # Cosine decay tốt hơn linear
    weight_decay=0.01,
    max_grad_norm = 1.0,
    save_strategy="epoch",
    save_total_limit=2,
    fp16=False,
    bf16=True,
    report_to="wandb",
    logging_steps=5,  # Log mỗi 5 steps
    optim="adamw_8bit",
    seed = 42,
)

In [8]:
# Step 6: Initialize SFTTrainer for supervised fine-tuning with EarlyStoppingCallback
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset = formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/2819 [00:00<?, ? examples/s]

In [9]:
wandb.login()
wandb.init(project="demo-colab", name="demo9") #122e74ba5f2e5ab0922497dad7da9ee2f78fe1d4

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 23020356 (23020356-vnu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


In [10]:
# Step 7: Train the model
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,819 | Num Epochs = 2 | Total steps = 231
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 29,933,568 of 3,115,872,256 (0.96% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,0.903600
10,0.935600
15,0.888500
20,0.854900
25,0.867300
30,0.832200
35,0.746400
40,0.665600
45,0.627200
50,0.598800


TrainOutput(global_step=231, training_loss=0.40670662163655996, metrics={'train_runtime': 1825.4517, 'train_samples_per_second': 2.008, 'train_steps_per_second': 0.127, 'total_flos': 4.082760850754765e+16, 'train_loss': 0.40670662163655996, 'epoch': 1.3063829787234043})

In [11]:
save_dir = "/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora"

# Save LoRA checkpoint
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)


('/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/tokenizer_config.json',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/special_tokens_map.json',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/chat_template.jinja',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/vocab.json',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/merges.txt',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/added_tokens.json',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/tokenizer.json')

In [12]:
import re
import os
from pathlib import Path
from tqdm import tqdm

# Load LoRA model
save_dir = "/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora"
model, tokenizer = FastLanguageModel.from_pretrained(
    save_dir,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
)

PROBLEMS_DIR = "/content/drive/MyDrive/QwenNoQuen/test"
OUTPUT_DIR = "/content/drive/MyDrive/QwenNoQuen/solution"
os.makedirs(OUTPUT_DIR, exist_ok=True)

problem_files = list(Path(PROBLEMS_DIR).glob("problem*.md"))
print(f"📁 Tìm thấy {len(problem_files)} bài toán\n")

def clean_code(text):
    """
    Xóa các phần đánh dấu markdown code block và các thẻ đặc biệt
    """
    # Xóa ```python ở đầu
    text = re.sub(r'^```python\s*', '', text, flags=re.MULTILINE)

    # Xóa ```<|im_end|> ở cuối
    text = re.sub(r'```<\|im_end\|>\s*$', '', text, flags=re.MULTILINE)

    # Xóa ``` đơn thuần (nếu có)
    text = re.sub(r'^```\s*', '', text, flags=re.MULTILINE)
    text = re.sub(r'```\s*$', '', text, flags=re.MULTILINE)

    # Xóa <|im_end|> đơn thuần (nếu có)
    text = re.sub(r'<\|im_end\|>', '', text)

    # Xóa khoảng trắng thừa ở đầu và cuối
    text = text.strip()

    return text

def extract_assistant_output(full):
    """
    Lấy phần assistant DUY NHẤT, bỏ hệ thống + user prompt.
    """
    if "<|im_start|>assistant" in full:
        return full.split("<|im_start|>assistant")[-1].strip()
    # fallback if no special tokens
    return full.strip()

for problem_file in problem_files:
    # đọc đề bài
    with open(problem_file, 'r', encoding='utf-8') as f:
        problem_content = f.read()

    # 1. Lấy tên file (problemX.md)
    filename = os.path.basename(problem_file)

    # 2. Loại bỏ phần mở rộng (.md)
    filename_without_ext = os.path.splitext(filename)[0] # Kết quả: problemx

    # 3. Loại bỏ tiền tố "problem"
    result_index = filename_without_ext.strip('problem') # Kết quả: 39

    # ====== ĐÚNG CHUẨN: TẠO CHAT TEMPLATE ======
    messages = [
        {"role": "system", "content": "You are a helpful coding assistant."},
        {"role": "user", "content": problem_content +
         "\n\nUse knowledge of data structures and algorithms. Think step by step to solve the code. Please provide ONLY Python function implementation, no explanation needed."}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Run model
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.1,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # ====== TÁCH RIÊNG PHẦN HÀM PYTHON ======
    assistant_only = extract_assistant_output(decoded)

    # ====== THAY ĐỔI: LƯU THÀNH FILE .py ======
    output_file = os.path.join(OUTPUT_DIR, f"solution{result_index}.py")
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(clean_code(assistant_only))
    print(f"Đã giải bài thứ: {result_index}")
print(f"\n✅ Đã lưu {len(problem_files)} solutions tại: {OUTPUT_DIR}")

==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
📁 Tìm thấy 50 bài toán

Đã giải bài thứ: 10
Đã giải bài thứ: 41
Đã giải bài thứ: 15
Đã giải bài thứ: 16
Đã giải bài thứ: 42
Đã giải bài thứ: 48
Đã giải bài thứ: 1
Đã giải bài thứ: 11
Đã giải bài thứ: 28
Đã giải bài thứ: 49
Đã giải bài thứ: 43
Đã giải bài thứ: 13
Đã giải bài thứ: 32
Đã giải bài thứ: 45
Đã giải bài thứ: 47
Đã giải bài thứ: 37
Đã giải bài thứ: 20
Đã giải bài thứ: 24
Đã giải bài thứ: 19
Đã giải bài thứ: 3
Đã giải bài thứ: 7
Đã giải bài thứ: 36
Đã giải bài thứ: 46
Đã giải bài thứ: 22
Đã giải bài thứ: 31
Đã giải bài thứ: 30
Đã